<a href="https://colab.research.google.com/github/41371116h/PL-Repo./blob/main/HW3%E5%BE%85%E8%BE%A6%E6%B8%85%E5%96%AE%E8%88%87%E7%95%AA%E8%8C%84%E9%90%98%E7%B4%80%E9%8C%84(AI_Plan).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# ============================================
# ✅ 任務清單管理系統（Google Sheets + 番茄鐘 + AI Plan）
# ============================================
import gradio as gr
import pandas as pd
import datetime
import gspread
from google.colab import auth
from google.auth import default
import time
import random

# ====== Google Sheets 設定 ======
SHEET_URL = "https://docs.google.com/spreadsheets/d/1GScHTHISiioV89XO5twgGl-IpaYG-0nMwhLRTizSpNI/edit"
TASKS_SHEET = "工作表2"
PLAN_SHEET = "番茄鐘(Gemini PLAN)"

# ====== Google Sheets 認證與初始化 ======
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    gsheets = gc.open_by_url(SHEET_URL)

    try:
        task_ws = gsheets.worksheet(TASKS_SHEET)
    except gspread.WorksheetNotFound:
        task_ws = gsheets.add_worksheet(title=TASKS_SHEET, rows="200", cols="6")
        task_ws.append_row(["id", "任務名稱", "優先級", "預估時間", "到期日", "狀態"], value_input_option="USER_ENTERED")

    try:
        plan_ws = gsheets.worksheet(PLAN_SHEET)
    except gspread.WorksheetNotFound:
        plan_ws = gsheets.add_worksheet(title=PLAN_SHEET, rows="50", cols="4")
        plan_ws.append_row(["時間段", "任務名稱", "建議", "備註"], value_input_option="USER_ENTERED")

    print("✅ 已成功連線 Google Sheets")
except Exception as e:
    task_ws = None
    plan_ws = None
    print(f"❌ Google Sheets 連線失敗: {e}")

# ====== 工具函式 ======
def read_tasks():
    if not task_ws:
        return pd.DataFrame(columns=["id", "任務名稱", "優先級", "預估時間", "到期日", "狀態"])
    data = task_ws.get_all_values()
    if len(data) <= 1:
        return pd.DataFrame(columns=data[0] if data else [])
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

def write_tasks(df):
    if not task_ws:
        return
    task_ws.clear()
    task_ws.append_row(list(df.columns))
    for _, row in df.iterrows():
        task_ws.append_row(row.tolist(), value_input_option="USER_ENTERED")

def new_id():
    return datetime.datetime.now().strftime("%Y%m%d%H%M%S")

def refresh_all():
    df = read_tasks()
    task_choices = df["任務名稱"].tolist() if not df.empty else []
    return df, task_choices

# ====== 任務操作功能 ======
def add_task(name, priority, est_min, due_date):
    if not task_ws:
        return "❌ Sheets 未連線", read_tasks()
    try:
        df = read_tasks()
        new_row = {
            "id": new_id(),
            "任務名稱": name,
            "優先級": priority,
            "預估時間": str(est_min),
            "到期日": due_date,
            "狀態": "todo"
        }
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        write_tasks(df)
        df, _ = refresh_all()
        return f"✅ 已新增任務：{name}", df
    except Exception as e:
        return f"❌ 新增任務錯誤：{e}", read_tasks()

def update_status(task_name, new_status):
    if not task_ws:
        return "❌ Sheets 未連線", read_tasks()
    df = read_tasks()
    if task_name not in df["任務名稱"].values:
        return f"❌ 任務不存在：{task_name}", df
    df.loc[df["任務名稱"] == task_name, "狀態"] = new_status
    write_tasks(df)
    df, _ = refresh_all()
    return f"✏️ 任務「{task_name}」狀態更新為 {new_status}", df

def delete_task(task_name):
    if not task_ws:
        return "❌ Sheets 未連線", read_tasks()
    df = read_tasks()
    if task_name not in df["任務名稱"].values:
        return f"❌ 任務不存在：{task_name}", df
    df = df[df["任務名稱"] != task_name]
    write_tasks(df)
    df, _ = refresh_all()
    return f"🗑️ 已刪除任務「{task_name}」", df

def mark_done(task_name):
    return update_status(task_name, "done")

def sync_tasks():
    df, choices = refresh_all()
    return df, choices

# ====== 番茄鐘 ======
timer_state = {"running": False, "paused": False, "start_time": None, "remaining": 0, "duration": 0}

def timer_generator(task_name, mode):
    timer_state["duration"] = 25*60 if mode=="work" else 5*60
    timer_state["remaining"] = timer_state["duration"]
    timer_state["running"] = True
    timer_state["paused"] = False
    timer_state["start_time"] = time.time()

    while timer_state["running"] and timer_state["remaining"] > 0:
        if timer_state["paused"]:
            timer_state["start_time"] = time.time() - (timer_state["duration"] - timer_state["remaining"])
            time.sleep(0.2)
            continue
        elapsed = time.time() - timer_state["start_time"]
        timer_state["remaining"] = timer_state["duration"] - elapsed
        if timer_state["remaining"] <= 0:
            timer_state["remaining"] = 0
            timer_state["running"] = False
        mins, secs = divmod(int(timer_state["remaining"]), 60)
        yield f"<h1 style='font-size:60px; color:#FF4500'>{task_name} ⏱ {mins:02d}:{secs:02d}</h1>"
        time.sleep(1)
    yield f"<h1 style='font-size:50px; color:green'>⏰ {task_name} 時間到！</h1>"

def pause_timer():
    timer_state["paused"] = not timer_state["paused"]
    status = "⏸ 暫停中" if timer_state["paused"] else "▶️ 繼續計時"
    return f"<h2>{status}</h2>"

def reset_timer(task_name, mode):
    timer_state["duration"] = 25*60 if mode=="work" else 5*60
    timer_state["remaining"] = timer_state["duration"]
    timer_state["running"] = False
    timer_state["paused"] = False
    mins, secs = divmod(int(timer_state["remaining"]), 60)
    return f"<h1 style='font-size:60px; color:#1E90FF'>{task_name} ⏱ {mins:02d}:{secs:02d}</h1>"

# ====== AI Plan 功能 ======
def generate_plan_with_ai(tasks):
    """
    模擬 AI 將任務分配到三個時間段，並給簡單建議
    """
    if not tasks:
        return pd.DataFrame(columns=["時間段","任務名稱","建議","備註"])

    random.shuffle(tasks)
    n = len(tasks)
    morning = tasks[:n//3]
    afternoon = tasks[n//3: 2*n//3]
    evening = tasks[2*n//3:]

    plan_data = []
    for t in morning:
        plan_data.append(["Morning", t, "優先完成重要任務", ""])
    for t in afternoon:
        plan_data.append(["Afternoon", t, "集中精力處理中等重要任務", ""])
    for t in evening:
        plan_data.append(["Evening", t, "輕鬆完成小任務或整理", ""])

    df_plan = pd.DataFrame(plan_data, columns=["時間段","任務名稱","建議","備註"])

    # 寫入 Google Sheets
    if plan_ws:
        plan_ws.clear()
        plan_ws.append_row(list(df_plan.columns))
        for _, row in df_plan.iterrows():
            plan_ws.append_row(row.tolist(), value_input_option="USER_ENTERED")

    return df_plan

# ====== Gradio 介面 ======
with gr.Blocks(title="任務清單管理系統") as demo:
    gr.Markdown("## ✅ 任務清單管理系統（Google Sheets 即時同步 + 番茄鐘 + AI Plan）")

    with gr.Tabs():
        # --------------------------
        # 任務清單
        # --------------------------
        with gr.Tab("📋 任務清單"):
            with gr.Row():
                task_name_input = gr.Textbox(label="任務名稱")
                priority_input = gr.Dropdown(["H","M","L"], value="M", label="優先級")
            with gr.Row():
                est_min_input = gr.Number(label="預估時間（分鐘）", value=25)
                due_date_input = gr.Textbox(label="到期日 (YYYY-MM-DD，可留空)")
            btn_add = gr.Button("➕ 新增任務")
            msg_add = gr.Markdown()
            btn_refresh = gr.Button("🔄 重新整理清單")
            grid_tasks = gr.Dataframe(value=read_tasks(), label="📋 任務清單", interactive=False)

        # --------------------------
        # 任務管理
        # --------------------------
        with gr.Tab("✏️ 任務管理"):
            df, choices = refresh_all()
            sel_task = gr.Dropdown(choices=choices, label="選擇任務")
            new_status = gr.Dropdown(["todo","in-progress","done"], label="更新狀態")
            btn_update = gr.Button("✏️ 更新狀態")
            btn_delete = gr.Button("🗑️ 刪除任務")
            btn_done = gr.Button("✅ 標記完成")
            btn_sync_select = gr.Button("🔄 同步選擇任務")
            msg_action = gr.Markdown()
            grid_tasks2 = gr.Dataframe(value=read_tasks(), label="目前任務", interactive=False)

        # --------------------------
        # 番茄鐘
        # --------------------------
        with gr.Tab("⏱️ 番茄鐘"):
            df, choices = refresh_all()
            sel_task_timer = gr.Dropdown(choices=choices, label="選擇任務計時")
            mode = gr.Radio(choices=["work","break"], value="work", label="模式選擇")
            btn_start_timer = gr.Button("▶️ Start")
            btn_pause_timer = gr.Button("⏸️ 暫停 / 繼續")
            btn_reset_timer = gr.Button("🔄 重置")
            timer_output = gr.HTML(label="倒數計時")

        # --------------------------
        # AI Plan
        # --------------------------
        with gr.Tab("🤖 AI Plan"):
            btn_generate_plan = gr.Button("🧠 生成今日計畫")
            plan_output = gr.Dataframe(value=pd.DataFrame(columns=["時間段","任務名稱","建議","備註"]), label="AI 行動計畫")

    # ====== 事件綁定 ======
    # 任務新增與刷新
    btn_add.click(add_task, inputs=[task_name_input, priority_input, est_min_input, due_date_input], outputs=[msg_add, grid_tasks])
    btn_refresh.click(lambda: read_tasks(), outputs=[grid_tasks])

    # 任務管理
    btn_update.click(update_status, inputs=[sel_task, new_status], outputs=[msg_action, grid_tasks2])
    btn_delete.click(delete_task, inputs=[sel_task], outputs=[msg_action, grid_tasks2])
    btn_done.click(mark_done, inputs=[sel_task], outputs=[msg_action, grid_tasks2])
    btn_sync_select.click(sync_tasks, outputs=[grid_tasks2, sel_task])

    # 番茄鐘
    btn_start_timer.click(timer_generator, inputs=[sel_task_timer, mode], outputs=[timer_output])
    btn_pause_timer.click(pause_timer, outputs=[timer_output])
    btn_reset_timer.click(reset_timer, inputs=[sel_task_timer, mode], outputs=[timer_output])

    # AI Plan
    btn_generate_plan.click(lambda: generate_plan_with_ai(read_tasks()["任務名稱"].tolist()), outputs=[plan_output])

# ====== 啟動 App ======
if __name__ == "__main__":
    demo.launch(debug=True, share=True)


✅ 已成功連線 Google Sheets
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://521ddfe3fa3feaa342.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://521ddfe3fa3feaa342.gradio.live
